# Data intgeration notebook

Use this notebook to update the database with your new data

## PostgreSQL connection check

Let's firt check that your database connectin works.

Adapt the credentials to your configuration and run this cell to make sure you can connect to your database.

In [26]:
POSTGRES_HOST = "localhost"
POSTGRES_PORT = "5432"
POSTGRES_DB = "hack_zurich"
POSTGRES_USER = "hack_zurich"
POSTGRES_PASSWORD = "hack_zurich"
INPUT_DATA_FOLDER = "input_data"

import psycopg2

db_connection = psycopg2.connect(
    dbname=POSTGRES_DB,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_HOST,
    port=POSTGRES_PORT)

db_cursor = db_connection.cursor()

db_cursor.execute("SELECT * FROM accessibility_bus LIMIT 50")

print("Connection to PostgreSQL works")

Connection to PostgreSQL works


# Data update

Run this cell to update your data (add new rows).

    print(f"""INSERT INTO indicators VALUES (
        {indicator_id},
        {name},
        {description},
        {source or None},
        {unit_short or None},
        {unit_long or None},
        {current_date or None},
        {min_year or None},
        {max_year or None}
    )""")

In [42]:
import csv

def read_csv(csv_file_path):
    with open(csv_file_path, "r") as csv_file:
        reader = csv.reader(csv_file)

        next(reader) # Remove the headers
        
        return list(reader)

    
# print(len(read_csv(f"{INPUT_DATA_FOLDER}/EN_INDICATORS.csv")))
for indicator_id, name, description, source, unit_short, unit_long, current_date, min_year, max_year in read_csv(f"{INPUT_DATA_FOLDER}/EN_INDICATORS.csv"):
    cursor.execute("""INSERT INTO indicators (indicator_id, name, description, source, unit_short, unit_long, current_date, min_year, max_year) VALUES (
        %(indicator_id)s,
        %(name)s,
        %(description)s,
        %(source)s,
        %(unit_short)s,
        %(unit_long)s,
        %(current_date)s,
        %(min_year)s,
        %(max_year)s
    )""", {
        'indicator_id': indicator_id,
        'name': name,
        'description': description,
        'source': source or None,
        'unit_short': unit_short or None,
        'unit_long': unit_long or None,
        'current_date': current_date or None,
        'min_year': min_year or None,
        'max_year': max_year or None
    })
    

SyntaxError: syntax error at or near "current_date"
LINE 1: ...name, description, source, unit_short, unit_long, current_da...
                                                             ^


## Views creation

Run he below cell to generate the SQL code necessary to create the views.

In [23]:
import csv

with open(f"{INPUT_DATA_FOLDER}/INDICATORS_VIEWS.csv", "r") as views_csv:
    reader = csv.reader(views_csv)

    next(reader) # Remove the headers

    for indicator_id, short_description, question_type, view_name, view_column_value in reader:
        # TODO: maybe use 'CREATE OR REPLACE VIEW' instead ?
        print(f"""CREATE VIEW {view_name} AS
SELECT v.value as "{view_column_value}", v.year, v.spatialunit_id
FROM indicators i inner join indicator_values2 v on i.indicator_id = v.indicator_id
WHERE trim(i.name) = '{short_description}';

""")

CREATE VIEW accessibility_bus AS
SELECT v.value as "access_by_bus", v.year, v.spatialunit_id
FROM indicators i inner join indicator_values2 v on i.indicator_id = v.indicator_id
WHERE trim(i.name) = 'share of people living in proximity of a busstop';


CREATE VIEW accessibility_train AS
SELECT v.value as "access_by_suburban_train", v.year, v.spatialunit_id
FROM indicators i inner join indicator_values2 v on i.indicator_id = v.indicator_id
WHERE trim(i.name) = 'share of people living in proximity of a trainstation';


CREATE VIEW number_of_passenger_cars AS
SELECT v.value as "passenger_cars_per_1000_inhabitants", v.year, v.spatialunit_id
FROM indicators i inner join indicator_values2 v on i.indicator_id = v.indicator_id
WHERE trim(i.name) = 'number of cars per capita';


CREATE VIEW distance_next_stop AS
SELECT v.value as "distance_to_next_stop", v.year, v.spatialunit_id
FROM indicators i inner join indicator_values2 v on i.indicator_id = v.indicator_id
WHERE trim(i.name) = 'average dist